## Video Preview Creator (adapted for VIS 2021)

Creates a custom title slide for each author submitted video and puts it in front of the submitted video (approx. 5 sec title + 25 sec video)

### Requirements

system installs:


cairo : check https://cairocffi.readthedocs.io/en/stable/overview.html#installing-cairocffi

ffmpeg: should be installed somewhere in the system path (such that running `ffmpeg` from the command line works.)

*make sure they are in the system path


And all the following python packages

In [1]:
import pandas as pd

import os
import sys
import click
import cairocffi as cairo
import subprocess

import subprocess
import yaml
import wave
import os
import re

#### Redapted from the previous year script

In [2]:
def probe( filename ):
    '''get some information about a video file'''

    proc = subprocess.Popen( [
            'ffprobe', 
            '-show_format',
            '-show_streams',
            '-print_format', 'json',
            '-i', filename
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE )

    out, err = proc.communicate()

    if proc.returncode:
        return None

    raw_info = yaml.load(out)

    info          = dict()                
    info['video'] = dict()
    info['audio'] = dict()

    info['duration'] = float( raw_info['format']['duration'] )
    info['size']     = float( raw_info['format']['size'] ) / 1048576.

    for s in raw_info['streams']:

        if s['codec_type'] == 'video':
            #print(s)
            info['video']['codec']  = s['codec_name']
            info['video']['format'] = s['pix_fmt']
            info['video']['width']  = int( s['width'] )
            info['video']['height'] = int( s['height'] )
            #info['video']['aspect'] = s['display_aspect_ratio']
            info['video']['fps']    = s['r_frame_rate']
            
        elif s['codec_type'] == 'audio':
            
            info['audio']['codec'] = s['codec_name']

    if 'codec' not in info['audio']:
        info['audio']['codec'] = '-'

    return info


In [3]:
def generate_vp(title_data):
   
    final = cairo.ImageSurface(cairo.FORMAT_ARGB32, 1920, 1080)
    ctx = cairo.Context(final)

    def linebreak(text, max_width, sep=' '):
        lines = []

        for text in text.split("\\n"):
            tokens = text.split(sep)
            line = []

            for token in tokens:
                extline = line + [token]
                width = ctx.text_extents(sep.join(extline))[2]

                if width > max_width:
                    lines.append((sep.join(line) + sep))
                    line = [token]
                else:
                    line = extline

            lines.append(sep.join(line))

        return [l.strip() for l in lines]

    def draw_text(text, x, y, size):

        ctx.set_font_size(size)

        y += ctx.font_extents()[0]

        for num, line in enumerate(text):
            ctx.move_to(x, y)
            ctx.show_text(line)
            y = y + 1.2 * size

    img_bkgnd = cairo.ImageSurface.create_from_png( 'preview-background 2021_yellow.png' )

    # clear image
    ctx.rectangle( 0, 0, 1920, 1080 )
    ctx.set_source_rgba( 1, 1, 1, 1 )
    ctx.fill()

    ctx.set_source_surface( img_bkgnd )
    ctx.paint()

    # draw paper type
    ctx.select_font_face( "Futura Medium" )
    ctx.set_font_size( 30 )
    ctx.set_source_rgba( 1, 1, 1, 1 )

    draw_text( [title_data[0].upper()], 1080, 112, 50 )

    # draw title
    ctx.select_font_face( "Gotham Bold" )
    ctx.set_source_rgba( 1, 1, 1, 1 )

    for fontsize in range( 60, 50, -2 ):

        ctx.set_font_size( fontsize )
        lines = linebreak( title_data[1], 1800 )

        if len(lines) < 4:
            break

    if len(lines) > 2:
        ctx.translate( 0,  0.3*fontsize )
    if len(lines) < 1:
        ctx.translate( 0,  0.3*fontsize )

    draw_text( lines, 90, 420-1.2*fontsize*len(lines), fontsize )

    # draw authors
    ctx.select_font_face( "Gotham Book" )
    ctx.set_source_rgba( 1, 1, 1, 1 )

    for author_fontsize in range( 50, 40, -2 ):

        ctx.set_font_size( author_fontsize )
        lines = linebreak( title_data[2], 1800, ',' )

        if len(lines) < 4:
            break

    draw_text( lines, 90, 480, 35 )
    
    # draw awards information
    if(title_data[6] == 'Best Paper' or title_data[6] == 'Honarable Mentions'):
        ctx.select_font_face( "Gotham Bold" )
        ctx.set_source_rgba( 1, 1, 1, 1 )

        for session_fontsize in range( 60, 50, -2 ):

            ctx.set_font_size( session_fontsize )
            lines = linebreak( '*'+title_data[6], 1800, ',' )

            if len(lines) < 4:
                break

        draw_text( lines, 90, 540, 40 )
    
    # draw session info
    ctx.select_font_face( "Gotham Bold" )
    ctx.set_source_rgba( 1, 1, 1, 1 )

    for session_fontsize in range( 80, 40, -2 ):

        ctx.set_font_size( session_fontsize )
        lines = linebreak( title_data[4], 1800, ',' )

        if len(lines) < 4:
            break

    draw_text( lines, 90, 700, 40 )

    png_file = 'tmp_yellow.png'

    final.write_to_png( png_file )
    
    # use FFMPEG to prepend image to video
    # filter syntax: https://ffmpeg.org/ffmpeg-filters.html#Filtergraph-syntax-1
    
    # check if preview already exists
    outfile = os.path.join( title_data[5] )

#     if os.path.exists( outfile ):
#         print( '  preview found in', outfile )
#         return outfile

    #input_file = find_video_file( meta['id'] )
    input_file = title_data[3]
    info = probe( input_file )
    
    filter  = '[0:v]trim=duration=5,fade=t=out:st=4.5:d=0.5[v0];'
    filter += 'aevalsrc=0:d=5[a0];'
    filter += '[1:v]scale=w=1920:h=1080[v1];'

    if info['audio']['codec'] == '-':
        filter += 'aevalsrc=0:d=25[a1];'
    else:
        filter += '[1:a]anull[a1];'

    filter += '[v0][a0][v1][a1]concat=n=2:v=1:a=1[v][a]'
    
    cmd = [
        'ffmpeg',
        '-v', 'error',
        '-framerate', info['video']['fps'],
        '-f', 'image2pipe',
        '-vcodec', 'png',
        '-loop', '1',
        '-i', png_file,
        '-i', input_file,
        '-filter_complex', filter,
        '-map', '[v]',
        '-map', '[a]',
        '-c:v', 'libx264',
        '-pix_fmt', 'yuv420p',
        '-profile:v', 'main',
        '-preset', 'slow',
        '-y', outfile
    ]

    try:
        p = subprocess.Popen( cmd )
        p.wait()
    except:
        # on error, delete the output file
        os.unlink( outfile )
        print( '  abort, deleted', outfile )
        raise


In [4]:
df = pd.read_excel(open('VP_2021_videodata.xlsx', 'rb'),sheet_name='VP_ShortPaper', engine='openpyxl')

In [5]:
df

,Submission ID,Type,Title,Authors,Contact Email,Abstract,Award,Filename,Session Name,Session Time
0,1012,Vis Short Paper,Towards a Survey on Static and Dynamic Hypergr...,"Maximilian T. Fischer, Alexander Frings, Danie...",max.fischer@uni-konstanz.de,Leveraging hypergraph structures to model adva...,NaN,v-short_1012_Fischer_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
1,1013,Vis Short Paper,CloudFindr: A Deep Learning Cloud Artifact Mas...,"Kalina Borkiewicz, Viraj Shah, J.P. Naiman, Ch...",kalina@illinois.edu,Artifact removal is an integral component of c...,NaN,cloudfindr.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
2,1016,Vis Short Paper,An Exploration And Validation of Visual Factor...,"Jun Yuan, Oded Nov, Enrico Bertini",junyuan@nyu.edu,Rule sets are often used in Machine Learning (...,NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810
3,1024,Vis Short Paper,Fast & Accurate Gaussian Kernel Density Estima...,Jeffrey Heer,jheer@uw.edu,Kernel density estimation (KDE) models a discr...,NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810
4,1025,Vis Short Paper,TimeElide: Visual Analysis of Non-Contiguous T...,"Michael Oppermann, Luce Liu, Tamara Munzner",opperman@cs.ubc.ca,We introduce the design and implementation of ...,NaN,v-short_1025_Oppermann_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
5,1030,Vis Short Paper,Time-Varying Fuzzy Contour Trees,"Anna-Pia Lohfink, Frederike Gartzky, Florian W...",lohfink@cs.uni-kl.de,"We present a holistic, topology-based visualiz...",NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810
6,1032,Vis Short Paper,A Visual Analytics System for Water Distributi...,"Yiran Li, Erin Musabandesu, Takanori Fujiwara,...",ranli@ucdavis.edu,The optimization of water distribution systems...,NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810
7,1033,Vis Short Paper,Gemini²: Generating Keyframe-Oriented Animated...,"Younghoon Kim, Jeffrey Heer",yhkim01@cs.washington.edu,Complex animated transitions may be easier to ...,NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810
8,1034,Vis Short Paper,On The Potential of Zines as a Medium for Visu...,Andrew M McNutt,mcnutt@uchicago.edu,Zines are a form of small-circulation self-pro...,NaN,v-short_1034_McNutt_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
9,1035,Vis Short Paper,"CellProfiler Analyst Web (CPAW) - Exploration,...","Bella baidak, Yahiya Hussain, Emma Kelminson, ...",bella.baidak001@umb.edu,CellProfiler Analyst has enabled the scientifi...,NaN,NaN,"Social Sciences, Software Tools, Journalism, a...",0800-0810


In [41]:
#remove entries with no files. probably authors did not submit
clean_df = df.loc[df['Filename'].notna()]

In [42]:
clean_df

,Submission ID,Type,Title,Authors,Contact Email,Abstract,Award,Filename,Session Name,Session Time
0,1012,Vis Short Paper,Towards a Survey on Static and Dynamic Hypergr...,"Maximilian T. Fischer, Alexander Frings, Danie...",max.fischer@uni-konstanz.de,Leveraging hypergraph structures to model adva...,NaN,v-short_1012_Fischer_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
1,1013,Vis Short Paper,CloudFindr: A Deep Learning Cloud Artifact Mas...,"Kalina Borkiewicz, Viraj Shah, J.P. Naiman, Ch...",kalina@illinois.edu,Artifact removal is an integral component of c...,NaN,cloudfindr.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
4,1025,Vis Short Paper,TimeElide: Visual Analysis of Non-Contiguous T...,"Michael Oppermann, Luce Liu, Tamara Munzner",opperman@cs.ubc.ca,We introduce the design and implementation of ...,NaN,v-short_1025_Oppermann_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
8,1034,Vis Short Paper,On The Potential of Zines as a Medium for Visu...,Andrew M McNutt,mcnutt@uchicago.edu,Zines are a form of small-circulation self-pro...,NaN,v-short_1034_McNutt_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
10,1047,Vis Short Paper,GeoSneakPique: Visual autocompletion for geosp...,"Vidya Setlur, Sarah Battersby, Tracy Kam Hung ...",vsetlur@tableau.com,How many crimes occurred in the city center? A...,NaN,v-short_1047_Setlur_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
11,1048,Vis Short Paper,Semantic Resizing of Charts Through Generaliza...,"Vidya Setlur, Haeyong Chung",vsetlur@tableau.com,Inspired by cartographic generalization princi...,Honarable Mentions,v-short_1048_Setlur_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
14,1064,Vis Short Paper,Atlas: Grammar-based Procedural Generation of ...,"Zhicheng Liu, Chen Chen, Francisco Morales, Yi...",leozcliu@umd.edu,"We present Atlas, a procedural grammar for con...",NaN,v-short_1064_Liu_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
15,1065,Vis Short Paper,"""Why did my AI agent lose?"": Visual Analytics ...","Delyar Tabatabai, Anita Ruangrotsakun, Jed Irv...",minsuk.kahng@oregonstate.edu,How can we help domain-knowledgeable users who...,NaN,v-short_1065_Tabatabai_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
16,1072,Vis Short Paper,Inspecting the Process of Bank Credit Rating v...,"Qiangqiang Liu, Tangzhi Ye, Zhihua Zhu, Xiaoju...",qliba@connect.ust.hk,Bank credit rating refers to classify commerci...,NaN,v-short_ 1072_Qiangqiang_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810
18,1082,Vis Short Paper,Bayesian Modelling of Alluvial Diagram Complexity,"Anjana Arunkumar, Shashank Ginjpalli, Chris Bryan",aarunku5@asu.edu,Alluvial diagrams are a popular technique for ...,NaN,v-short_1082_Arunkumar_Preview.mp4,"Social Sciences, Software Tools, Journalism, a...",0800-0810


In [43]:
print('Available videos:', len(clean_df))
print('Total Submissions:', len(df))


Available videos: 17
Total Submissions: 41


In [44]:
input_vid_dir = 'video_dir/VIS Short/'
output_vid_dir = 'video_dir/VP_out_VIS Short/'

In [47]:
for index, row in clean_df.iterrows():

    video_metadata = [row['Type'],
                      row['Title'],
                      row['Authors'],
                      input_vid_dir + row['Filename'],
                      row['Session Name'] + ': ' + row['Session Time'],
                      #'Social Sciences, Software Tools, Journalism, and Storytelling: Friday, 0815 - 0830',
                      output_vid_dir + row['Filename'],
                      row['Award']
                     ]
    
    print(index, video_metadata, '\n\n')
    generate_vp(video_metadata)
    
    
    

0 ['Vis Short Paper', 'Towards a Survey on Static and Dynamic Hypergraph Visualizations', 'Maximilian T. Fischer, Alexander Frings, Daniel Keim, Daniel Seebacher', 'video_dir/VIS Short/v-short_1012_Fischer_Preview.mp4', 'Social Sciences, Software Tools, Journalism, and Storytelling: 0800-0810', 'video_dir/VP_out_VIS Short/v-short_1012_Fischer_Preview.mp4', nan] 


1 ['Vis Short Paper', 'CloudFindr: A Deep Learning Cloud Artifact Masker for Satellite DEM Data', 'Kalina Borkiewicz, Viraj Shah, J.P. Naiman, Chuanyue Shen, Stuart Levy, Jeffrey Carpenter', 'video_dir/VIS Short/cloudfindr.mp4', 'Social Sciences, Software Tools, Journalism, and Storytelling: 0800-0810', 'video_dir/VP_out_VIS Short/cloudfindr.mp4', nan] 


4 ['Vis Short Paper', 'TimeElide: Visual Analysis of Non-Contiguous Time Series Slices', 'Michael Oppermann, Luce Liu, Tamara Munzner', 'video_dir/VIS Short/v-short_1025_Oppermann_Preview.mp4', 'Social Sciences, Software Tools, Journalism, and Storytelling: 0800-0810', 'vide